# Agriculture AI Agent API - Test Cases

Notebook để test các trường hợp khác nhau của Agriculture AI Agent API.

## Mục tiêu
- Test các loại câu hỏi thời tiết
- Test các câu hỏi nông nghiệp cơ bản
- Test các câu hỏi kỹ thuật nông nghiệp
- So sánh kết quả thực tế vs kỳ vọng

## API Endpoint
API được deploy tại: [YOUR_API_ENDPOINT]

In [1]:
# Import Required Libraries
import requests
import json
import time
from datetime import datetime
import pandas as pd
from IPython.display import display, JSON, Markdown

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Define API Endpoint and Helper Functions

# API Configuration
API_ENDPOINT = "[YOUR_API_ENDPOINT]"  # Thay thế bằng endpoint thực tế
HEALTH_CHECK_URL = f"{API_ENDPOINT}/"
QUERY_URL = f"{API_ENDPOINT}/query"

# Headers for requests
HEADERS = {
    "Content-Type": "application/json"
}

def send_query(user_query: str, timeout: int = 30) -> dict:
    """
    Gửi query tới Agriculture AI Agent API
    
    Args:
        user_query (str): Câu hỏi từ người dùng
        timeout (int): Timeout cho request (seconds)
    
    Returns:
        dict: Response từ API hoặc error info
    """
    try:
        print(f"\n🔄 Đang gửi query: {user_query[:50]}...")
        
        # Prepare request data
        data = {"user_query": user_query}
        
        # Record start time
        start_time = time.time()
        
        # Send request
        response = requests.post(
            QUERY_URL,
            headers=HEADERS,
            json=data,
            timeout=timeout
        )
        
        # Calculate response time
        response_time = time.time() - start_time
        
        # Check status code
        response.raise_for_status()
        
        # Parse JSON response
        result = response.json()
        
        print(f"✅ Response nhận được trong {response_time:.2f}s")
        
        return {
            "success": True,
            "query": user_query,
            "response": result.get("agent_response", ""),
            "response_time": response_time,
            "status_code": response.status_code,
            "timestamp": datetime.now().isoformat()
        }
        
    except requests.exceptions.Timeout:
        print(f"⏰ Timeout sau {timeout}s")
        return {
            "success": False,
            "query": user_query,
            "error": "Timeout",
            "response_time": timeout,
            "timestamp": datetime.now().isoformat()
        }
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return {
            "success": False,
            "query": user_query,
            "error": str(e),
            "timestamp": datetime.now().isoformat()
        }
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON decode error: {e}")
        return {
            "success": False,
            "query": user_query,
            "error": f"JSON decode error: {e}",
            "timestamp": datetime.now().isoformat()
        }
        
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return {
            "success": False,
            "query": user_query,
            "error": f"Unexpected error: {e}",
            "timestamp": datetime.now().isoformat()
        }

def health_check() -> dict:
    """
    Kiểm tra trạng thái của API
    """
    try:
        print("🔍 Đang kiểm tra trạng thái API...")
        response = requests.get(HEALTH_CHECK_URL, timeout=10)
        response.raise_for_status()
        
        result = response.json()
        print(f"✅ API đang hoạt động: {result}")
        return {
            "success": True,
            "status": result,
            "status_code": response.status_code
        }
        
    except Exception as e:
        print(f"❌ Health check failed: {e}")
        return {
            "success": False,
            "error": str(e)
        }

def display_result(result: dict):
    """
    Hiển thị kết quả test một cách đẹp mắt
    """
    if result["success"]:
        print(f"\n📝 Query: {result['query']}")
        print(f"⏱️  Response time: {result['response_time']:.2f}s")
        print(f"📅 Timestamp: {result['timestamp']}")
        print("\n🤖 Agent Response:")
        print("-" * 50)
        print(result['response'])
        print("-" * 50)
    else:
        print(f"\n❌ Error for query: {result['query']}")
        print(f"Error: {result.get('error', 'Unknown error')}")
        print(f"Timestamp: {result['timestamp']}")

In [3]:
# Health Check - Kiểm tra trạng thái API
print("=" * 60)
print("🏥 HEALTH CHECK")
print("=" * 60)

health_result = health_check()
display(JSON(health_result))

🏥 HEALTH CHECK
🔍 Đang kiểm tra trạng thái API...
✅ API đang hoạt động: {'status': 'healthy', 'message': 'Welcome to the Agriculture AI Agent API!'}
✅ API đang hoạt động: {'status': 'healthy', 'message': 'Welcome to the Agriculture AI Agent API!'}


<IPython.core.display.JSON object>

## 🌤️ Test Cases: Câu hỏi Thời tiết

Test các câu hỏi liên quan đến dự báo thời tiết và khí hậu.

In [4]:
# Weather Queries Test Cases
weather_queries = [
    "Nhiệt độ trung bình trong vòng 1 tuần tới tại Quảng Ngãi là bao nhiêu?",
    "Dự báo thời tiết 3 ngày tới tại TP.HCM",
    "Nhiệt độ trung bình tuần này ở Đà Nẵng",
    "Tình hình mưa bão tuần tới ở miền Trung"
]

print("=" * 60)
print("🌤️ WEATHER QUERIES TEST")
print("=" * 60)

weather_results = []

for i, query in enumerate(weather_queries, 1):
    print(f"\n--- Test Case {i}/{len(weather_queries)} ---")
    result = send_query(query)
    weather_results.append(result)
    display_result(result)
    
    # Delay between requests
    time.sleep(1)

🌤️ WEATHER QUERIES TEST

--- Test Case 1/4 ---

🔄 Đang gửi query: Nhiệt độ trung bình trong vòng 1 tuần tới tại Quản...
✅ Response nhận được trong 120.54s

📝 Query: Nhiệt độ trung bình trong vòng 1 tuần tới tại Quảng Ngãi là bao nhiêu?
⏱️  Response time: 120.54s
📅 Timestamp: 2025-05-30T22:25:34.841786

🤖 Agent Response:
--------------------------------------------------
Nhiệt độ trung bình trong vòng 1 tuần tới tại Quảng Ngãi, bắt đầu từ ngày 31/05/2025, dự kiến là khoảng 32.1°C, với thời tiết chủ đạo là ít mây và nắng nóng[1].

## Dự báo chi tiết nhiệt độ và thời tiết 7 ngày tới
Dưới đây là dự báo nhiệt độ cao nhất và thấp nhất hàng ngày tại Quảng Ngãi cho 7 ngày tới, bắt đầu từ 31/05/2025[1]:
- **Thứ Bảy (31/05/2025):** Nhiệt độ 36°C / 28°C, ít mây, trời nắng nóng[1].
- **Chủ Nhật (01/06/2025):** Nhiệt độ 38°C / 27°C, ít mây, trời nắng nóng gay gắt[1].
- **Thứ Hai (02/06/2025):** Nhiệt độ 38°C / 27°C, ít mây, trời nắng nóng gay gắt[1].
- **Thứ Ba (03/06/2025):** Nhiệt độ 38°C / 28°C,

: 

: 

## 🌾 Test Cases: Câu hỏi Nông nghiệp Cơ bản

Test các câu hỏi về kỹ thuật trồng trọt và chăn nuôi cơ bản.

In [ ]:
# Basic Agriculture Queries Test Cases
agriculture_queries = [
    "Cách trồng rau muống trong thùng xốp",
    "Cách trồng lúa trong mùa khô?",
    "Kỹ thuật trồng rau sạch tại nhà",
    "Cách chăm sóc cây ớt trong mùa mưa",
]

print("=" * 60)
print("🌾 AGRICULTURE QUERIES TEST")
print("=" * 60)

agriculture_results = []

for i, query in enumerate(agriculture_queries, 1):
    print(f"\n--- Test Case {i}/{len(agriculture_queries)} ---")
    result = send_query(query)
    agriculture_results.append(result)
    display_result(result)
    
    # Delay between requests
    time.sleep(1)

## ⚙️ Test Cases: Câu hỏi Kỹ thuật Nông nghiệp

Test các câu hỏi về kỹ thuật nông nghiệp nâng cao, công nghệ và tính toán chuyên môn.

In [ ]:
# Technical Agriculture Queries Test Cases
technical_queries = [
    "Cách tính lượng phân bón cho 1 hecta lúa",
    "Tính toán liều lượng thuốc trừ sâu cho 1000m2 lúa",
    "Hệ thống IoT trong nông nghiệp thông minh",
    "Công nghệ blockchain trong chuỗi cung ứng nông sản"
]

print("=" * 60)
print("⚙️ TECHNICAL AGRICULTURE QUERIES TEST")
print("=" * 60)

technical_results = []

for i, query in enumerate(technical_queries, 1):
    print(f"\n--- Test Case {i}/{len(technical_queries)} ---")
    result = send_query(query)
    technical_results.append(result)
    display_result(result)
    
    # Delay between requests
    time.sleep(1)

## 🧪 Test Cases: Câu hỏi Biên giới và Xử lý Lỗi

Test các trường hợp đặc biệt để kiểm tra khả năng xử lý của hệ thống.

In [ ]:
# Edge Cases and Error Handling Test Cases
edge_case_queries = [
    # Non-agriculture queries (should be declined)
    "Cách nấu phở ngon",
    "Giá bitcoin hôm nay",
    
    # Mixed language queries
    "How to grow rice in Vietnam?",
    "Cách trồng rice trong mùa wet season",
    
    # Very short queries
    "Lúa",
    "Thời tiết",
    
    # Very long queries
    "Tôi muốn hỏi về việc trồng lúa trong điều kiện khí hậu thay đổi, cụ thể là làm thế nào để có thể áp dụng các kỹ thuật canh tác hiện đại, sử dụng phân bón hữu cơ và vô cơ một cách hợp lý, đồng thời kiểm soát sâu bệnh hiệu quả mà không gây hại đến môi trường, và liệu có thể áp dụng công nghệ IoT để giám sát và tự động hóa quy trình tưới tiêu không?",
    
    # Empty or invalid queries
    "",
    "   ",
    
    # Personal questions about the AI
    "Bạn là ai?",
    "Agrimini có thể làm gì?"
]

print("=" * 60)
print("🧪 EDGE CASES AND ERROR HANDLING TEST")
print("=" * 60)

edge_case_results = []

for i, query in enumerate(edge_case_queries, 1):
    print(f"\n--- Edge Case {i}/{len(edge_case_queries)} ---")
    if query.strip() == "":
        print(f"Query: [EMPTY STRING]")
    else:
        print(f"Query: {query[:100]}{'...' if len(query) > 100 else ''}")
    
    result = send_query(query)
    edge_case_results.append(result)
    display_result(result)
    
    # Delay between requests
    time.sleep(1)

## 📊 Tổng kết và Phân tích Kết quả

Phân tích tổng quan về hiệu suất và chất lượng phản hồi của API.

In [ ]:
# Tổng hợp và phân tích kết quả
all_results = weather_results + agriculture_results + technical_results + edge_case_results

# Tạo DataFrame để phân tích
analysis_data = []
for result in all_results:
    analysis_data.append({
        'Query Type': (
            'Weather' if result in weather_results else
            'Agriculture' if result in agriculture_results else
            'Technical' if result in technical_results else
            'Edge Case'
        ),
        'Query': result['query'][:50] + '...' if len(result['query']) > 50 else result['query'],
        'Success': result['success'],
        'Response Time (s)': result.get('response_time', 0),
        'Response Length': len(result.get('response', '')) if result['success'] else 0,
        'Has Error': not result['success'],
        'Error Type': result.get('error', '') if not result['success'] else ''
    })

df_results = pd.DataFrame(analysis_data)

print("=" * 60)
print("📊 SUMMARY STATISTICS")
print("=" * 60)

# Thống kê tổng quan
print(f"Total Queries Tested: {len(all_results)}")
print(f"Successful Queries: {df_results['Success'].sum()}")
print(f"Failed Queries: {(~df_results['Success']).sum()}")
print(f"Success Rate: {df_results['Success'].mean()*100:.1f}%")

if df_results['Success'].any():
    successful_results = df_results[df_results['Success']]
    print(f"Average Response Time: {successful_results['Response Time (s)'].mean():.2f}s")
    print(f"Median Response Time: {successful_results['Response Time (s)'].median():.2f}s")
    print(f"Min Response Time: {successful_results['Response Time (s)'].min():.2f}s")
    print(f"Max Response Time: {successful_results['Response Time (s)'].max():.2f}s")
    print(f"Average Response Length: {successful_results['Response Length'].mean():.0f} characters")

# Thống kê theo loại query
print("\n📈 Success Rate by Query Type:")
success_by_type = df_results.groupby('Query Type')['Success'].agg(['count', 'sum', 'mean'])
success_by_type['Success Rate (%)'] = success_by_type['mean'] * 100
print(success_by_type[['count', 'sum', 'Success Rate (%)']])

# Hiển thị DataFrame
print("\n📋 Detailed Results:")
display(df_results)

In [ ]:
# Phân tích lỗi chi tiết
failed_results = df_results[~df_results['Success']]
if not failed_results.empty:
    print("\n❌ FAILED QUERIES ANALYSIS:")
    print("=" * 40)
    
    for idx, row in failed_results.iterrows():
        print(f"\nQuery: {row['Query']}")
        print(f"Error Type: {row['Error Type']}")
        print(f"Query Type: {row['Query Type']}")
        
    # Nhóm lỗi theo loại
    print("\n🔍 Error Types Summary:")
    error_summary = failed_results['Error Type'].value_counts()
    for error_type, count in error_summary.items():
        print(f"- {error_type}: {count} occurrence(s)")
else:
    print("\n✅ No failed queries! All tests passed successfully.")

## 🎯 Kiểm tra Chất lượng Phản hồi

Phân tích chất lượng và độ phù hợp của các phản hồi từ API.

In [ ]:
# Kiểm tra chất lượng phản hồi
def analyze_response_quality(results):
    """
    Phân tích chất lượng phản hồi dựa trên các tiêu chí:
    - Độ dài phản hồi
    - Có chứa citation không ([1], [2], etc.)
    - Có cấu trúc markdown không (##, -, *)
    - Ngôn ngữ phản hồi (tiếng Việt)
    """
    quality_analysis = []
    
    for result in results:
        if result['success']:
            response = result.get('response', '')
            
            # Kiểm tra các đặc điểm chất lượng
            has_citations = bool(re.search(r'\[\d+\]', response))
            has_markdown_headers = bool(re.search(r'^##', response, re.MULTILINE))
            has_bullet_points = bool(re.search(r'^- ', response, re.MULTILINE))
            has_vietnamese = bool(re.search(r'[àáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđ]', response.lower()))
            
            quality_analysis.append({
                'query': result['query'][:50] + '...' if len(result['query']) > 50 else result['query'],
                'response_length': len(response),
                'has_citations': has_citations,
                'has_markdown_headers': has_markdown_headers,
                'has_bullet_points': has_bullet_points,
                'has_vietnamese': has_vietnamese,
                'response_time': result.get('response_time', 0)
            })
    
    return pd.DataFrame(quality_analysis)

# Phân tích chất lượng cho từng loại query
print("=" * 60)
print("🎯 RESPONSE QUALITY ANALYSIS")
print("=" * 60)

# Import regex for pattern matching
import re

# Phân tích các query thành công
successful_results = [r for r in all_results if r['success']]

if successful_results:
    quality_df = analyze_response_quality(successful_results)
    
    print(f"\n📊 Quality Metrics Summary:")
    print(f"Responses with citations: {quality_df['has_citations'].sum()}/{len(quality_df)} ({quality_df['has_citations'].mean()*100:.1f}%)")
    print(f"Responses with markdown headers: {quality_df['has_markdown_headers'].sum()}/{len(quality_df)} ({quality_df['has_markdown_headers'].mean()*100:.1f}%)")
    print(f"Responses with bullet points: {quality_df['has_bullet_points'].sum()}/{len(quality_df)} ({quality_df['has_bullet_points'].mean()*100:.1f}%)")
    print(f"Responses in Vietnamese: {quality_df['has_vietnamese'].sum()}/{len(quality_df)} ({quality_df['has_vietnamese'].mean()*100:.1f}%)")
    
    print("\n📋 Quality Analysis Details:")
    display(quality_df)
    
else:
    print("\n❌ No successful responses to analyze.")